<a href="https://colab.research.google.com/github/dmanzanoa/WeatherAgent/blob/main/Weather_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install openai-agents
!pip install requests

In [2]:
%%capture
!pip install --no-cache-dir google-search-results

In [3]:
from serpapi.google_search import GoogleSearch
print("GoogleSearch imported directly!")

GoogleSearch imported directly!


In [5]:
import os
os.environ["OPENAI_API_KEY"] = "sxxxxx"  # your new API key here

In [8]:
import json
import asyncio
import nest_asyncio
from typing_extensions import TypedDict
import requests

from agents import Agent, FunctionTool, RunContextWrapper, function_tool, Runner

nest_asyncio.apply()  # <-- important for Jupyter/Colab

# Tool input: just the city name
class LocationByCity(TypedDict):
    city: str


@function_tool
async def get_weather(location: LocationByCity) -> str:
    """Fetch real-time weather for a given city using OpenWeatherMap API."""
    city = location["city"]
    response = requests.get(
        f"http://api.weatherapi.com/v1/current.json?key=b3c948ca8cb6495c87682750250808&q={city}")
        #f"&appid={OPENWEATHER_API_KEY}&units=metric"
    if response.status_code == 200:
        data = response.json()
        return data
    else:
      return f"Sorry, I couldn't get the weather for {city}."

@function_tool
async def get_events(location: LocationByCity) -> str:
    city = location["city"]
    params = {
      "engine": "google_events",
      "q": f"Events in {city}",
      "hl": "en",
      "gl": "us",
      "api_key": "xxxxx"
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    events_results = results["events_results"]
    return events_results

# Define the agent
weather_agent = Agent(
    name="Weather Agent",
    instructions=("""
    You are a helpful weather assistant. You can only answer questions about the weather of a city.

    If the user asks anything not related to the weather of a specific location, respond with:
    "I'm only able to provide weather information for cities. Please ask me about the weather in a specific place."

    If they ask about a location, call the `get_weather` function with the correct coordinates.
    """),
    tools=[get_weather, get_events],
)

# Define the agent
combined_agent = Agent(
    name="City Info Agent",
    instructions="""
    You are an assistant that can answer questions about:
    - weather in a city (use get_weather tool)
    - events happening in a city (use get_events tool)

    For weather questions, call the get_weather function with the city name.
    For events questions, call the get_events function with the city name.

    If the question is not about weather or events, reply:
    "Sorry, I can only provide information about weather or events in cities. Please ask about those."
    """,
    tools=[get_weather, get_events],
)

# Run test input
async def main():
    result1 = await Runner.run(combined_agent, input="What events are happening in New York?")
    print(result1.final_output)

await main()


Here are some events happening in New York:

1. **Noize MC**
   - **Date:** Sun, Aug 17, 5 – 9 PM
   - **Venue:** Terminal 5, 610 W 56th St #612, New York, NY
   - [Find Tickets](https://open.spotify.com/concert/6AU77Uwi4iBRJVWWjte0pb)

2. **Lost Together Festival**
   - **Date:** Sat, Aug 16, 5 PM – Sun, Aug 17, 1 AM
   - **Venue:** Industry City, 220 36th St, Brooklyn, NY
   - [Find Tickets](https://www.eventbrite.com/e/lost-together-festival-industry-city-nyc-tickets-1544495259899)

3. **New York Renaissance Faire**
   - **Date:** Fri, Aug 15, 8:00 PM – Sat, Aug 16, 12:30 PM
   - **Venue:** New York Renaissance Faire, 600 NY-17A, Tuxedo Park, NY
   - [Find Tickets](https://www.stubhub.com/_C-364823?pcid=PSUSAEVECONALLEF426C5CF9E&ps_p=8&ps_placement=eventfeed&ps=vn-150150643%7Cev-158447311%7Ccy-99326&ps_ev=158447311)

4. **Dark Star Orchestra**
   - **Date:** Fri, Aug 15, 6:00 – 7:30 PM
   - **Venue:** Pier 17, 89 South St, New York, NY
   - [Find Tickets](https://open.spotify.com/co

In [ ]:
async def main():
    # Weather question
    result1 = await Runner.run(combined_agent, input="What's the weather like in Paris?")
    print(result1.final_output)

    # Events question
    result2 = await Runner.run(combined_agent, input="What events are happening in New York?")
    print(result2.final_output)

    # Other question
    result3 = await Runner.run(combined_agent, input="Tell me about restaurants in Tokyo")
    print(result3.final_output)

await main()